In [1]:
import pandas as pd
import numpy as np
import requests, sys

Correct GWAS column dataset by moving chromosome coordinates to chr_id & chr_pos cols and replacing with rsIDs

In [ ]:
# filter for variants with no chromosomal coordinates and created a separated list of variants and dataframe
df = pd.read_table('TSVs/gwas_trimmed.tsv') 
df2 = df
df2 = df2.drop(df2[df2['rsid'].str.contains('rs')].index)
df2 = df2.reset_index()
df2 = df2.drop('index', axis=1)
variant_list = df2["rsid"].tolist() 

# create lists for API
chrom_list = []
chrom_pos_list = []
for variant in variant_list:
    variant_split = variant.split(":")
    chrom_list.append(variant_split[0].replace("chr", ""))
    chrom_pos_list.append(variant_split[1])
    
# assign coordinate values to correct columns of each SNP
for ind in df2.index:
    df2.loc[ind,'chr_pos'] = chrom_pos_list[ind]
    df2.loc[ind,'chr_id'] = chrom_list[ind]
    
# API for requesting variant information based on SNP chromosomal coordinates
def variant_search_API(chrom,chrom_pos):
    
    server = "https://rest.ensembl.org"
    ext = f"/overlap/region/human/{chrom}:{chrom_pos}-{chrom_pos}?feature=variation"

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    decoded = r.json()
    return decoded

# empty dataframe
variant_df = pd.DataFrame(columns=['rsid','chr_pos','chr_id']) 
# list for variants that cannot be found
invalid_variant = []
# loop to build dataframe
n = 0
for chrom,chrom_pos in zip(chrom_list,chrom_pos_list):
    n += 1
    try:
        decoded = variant_search_API(chrom,chrom_pos)
        rsID = decoded[0]['id']
        print(f"{n}-{rsID}") # print successfully retrieved SNPs
    except:
        invalid_variant.append(chrom + ":" + chrom_pos)
        print(f"{n}-{chrom}:{chrom_pos}") # print chromsomal coordinates which could not successfully retrieve SNPs
        continue
    variant_list = [rsID, chrom_pos, chrom]
    variant_row = pd.DataFrame(variant_list).T
    variant_row.columns = variant_df.columns
    variant_df = pd.concat([variant_df, variant_row])

invalid_variant = list(dict.fromkeys(invalid_variant)) # remove duplicates from list
print(f"Couldn't get rsids for variants: {invalid_variant}")   
# merge modified dataset with dataframe containing new SNP rsIDs
df2 = df2.drop('rsid', axis=1) # drop rsid column from modified dataset
df_filled = pd.merge(variant_df, df2, on=['chr_pos','chr_id']).drop_duplicates() # merge based on 'chr_pos' and 'chr_id' columns

# create another separate dataset with row that have rsID SNPs
df3 = df
df3 = df3.loc[df3['chr_pos'].notnull()]

# concat df originally containg SNP rsIDs with new df corrected with additional SNP rsIDs
df_corrected = pd.concat([df3, df_filled])
df_corrected = df_corrected[df_corrected["rsid"].str.contains("rs") == True] # remove any entries in the rsID column without rsIDs
# write out TSV file
df_corrected.chr_id = df_corrected.chr_id.astype(int) 
df_corrected.chr_pos = df_corrected.chr_pos.astype(int) 
df_corrected.to_csv('TSVs/T1D_GWAS_add.tsv', sep ='\t', index=False)

Add missing mapped gene entries

In [ ]:
df = pd.read_table('TSVs/T1D_GWAS_add.tsv', index_col = "i") # GWAS data 
df2 = df[df['mapped_gene'].isna()] # Filter for rows without mapped genes
SNP_list = df2['rsid'].to_list() # Extract list of SNPs

def add_mapped_gene(rsID):

    server = "https://rest.ensembl.org"
    ext = f"/variation/human/{rsID}?phenotypes=1"

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

    if not r.ok:
      r.raise_for_status()
      sys.exit()

    decoded = r.json()
    return decoded

# Find missing mapped genes and add to dataframe 
for SNP in SNP_list:
    decoded = add_mapped_gene(SNP) # request SNP data from Ensembl 
    # try to extract a mapped gene from dictionary
    try:
        mapped_gene = decoded['phenotypes'][0]['genes']
        SNP_ind = df2[df2['rsid']==SNP].index.values  # find SNP index in dataframe
        df.loc[SNP_ind,'mapped_gene'] = mapped_gene # replace empty cell in original dataframe with new mapped gene from SNP data
    except IndexError:
        continue
# write out corrected GWAS dataset as TSV
df.to_csv('TSVs/T1D_GWAS_add.tsv', sep ='\t')
print("Done")

Add cumulative chromosome positions

In [ ]:
df = pd.read_table('TSVs/T1D_GWAS_add.tsv') 

# -log the P-values and column to the table
df['-logp']= - np.log(df['p_value'])

# Put variants in order by max position in chromosome
running_pos = 0 # moves all integers down so first pos is 0
 
cumulative_pos = [] # create list of new series for position in whole genome

for chrom, group_df in df.groupby('chr_id'): # Group the region in each chromosome together
    cumulative_pos.append(group_df['chr_pos'] + running_pos) 
    running_pos+= group_df['chr_pos'].max() #tells us the last position in each chromosome

# Position of variant relative to whole chromosome, add column to the table    
df['cumulative_pos'] = pd.concat(cumulative_pos) 
# write out TSV file
df.to_csv('TSVs/T1D_GWAS_add.tsv', sep ='\t', index=False)